In [ ]:
% matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from math import factorial as fact

In [ ]:
# Parameters to fit:
# A: Boundary
# x: Stimulus strength
# k: Proportionality constant (Stim. str. = kx)
# tR: Residual time

# Equations to fit for each stimulus strength:
# pC = 1 / (1+exp(-2*A*k*abs(x)))
# mRT = A / (k*x) * tanh(A*k*x) + tR

# We can get approximate values for A, k and tR from Palmer et. al. '05
# Ranges of parameters to start with:
# A: 0.5 - 1
# k: 5 - 40
# tR: 0.25 - 0.5 (in seconds)
# x: 0 - 1

# To identify best fit, calculate the likelihood of predicted pC and mRT and find the maximum likelihood.

# Likelihood of pC follows a binomial distribution
# Lp = n! / (r!(n-r)! * pC(x)^r * (1-pC(x))^(n-r), where
# n = # trials, r = # required correct

# Likelihood of mRT follows a Gaussian distribution
# Lrt = 1 / (SDrt * (sqrt(2*pi))) * e^-((mRT(x) - oRT(x)) / SDrt)^2 * 1/2, where
# oRT = observed mRT, mRT = predicted mRT, SDrt = SD of predicted mRT
# VARrt = VARtd + VARtr, where
# VARtd = variance in decision time, VARtr = variance in residual time. Thus,
# VARrt = (A * tanh(A*k*x) - A*k*x * sech(A*k*x)) / (k*x)^3 + (0.1 * tR)^2

# Final fit measure is the log likelihood, which is the sum of the likelihoods of accuracy and mean RT, 
# over all combinations of coherence and distance
# Lprt = sigma(x)(ln(Lp(s)) + ln(Lrt(x)))

# The first pass of the model will be to estimate values of x without any assumptions about stimulus relationtips.
# The stopping point will be the point of least error.

In [ ]:
# Initialize A, k and tR parameters
A = np.linspace(1, 5, 10)
k = np.linspace(1, 15, 20)
tR = np.linspace(0.25, 1, 10)

As, ks, tRs = np.meshgrid(A, k, tR)
As = As.flatten()
ks = ks.flatten()
tRs = tRs.flatten()

# This is the overall number of permutations of A, k and tR being performed
nPar = len(As)

# Initialize stimulus strength parameter
x = np.linspace(0.001,0.2,15)

# Initialize the array that holds likelihood values
# Error = |Actual PC - Expected PC| + |Actual RT - Expected RT|
errs = np.ones((nPar, len(x), 20, 9)) * -9

In [ ]:
print(A)
print(k)
print(tR)
print(x)

In [ ]:
# List subjects to fit
subs = ['Sub01', 'Sub02', 'Sub04', 'Sub05', 'Sub06', 'Sub08', 'Sub10', 'Sub11', 'Sub13']
nSub = len(subs)

In [ ]:
# Initialize arrays to hold PC, mean and SD of RT, and # trials 
# for each coherence-distanct combination
# These values are obtained from the .csv files
pCs = np.zeros((20, nSub))
mRTs = np.zeros((20, nSub))
sdRTs = np.zeros((20, nSub))
Ns = np.zeros((20, nSub))

# Initialise a variable to hold # correct trials
# This will be computed from Ns and pCs
Rs = np.zeros((20, nSub))

In [ ]:
# Extract behavioral data (PC, mean and SD of RT, # trials) from csv files
for si in range(nSub):
    csvFile = '../Data/Behavior/' + subs[si] + '_behavData.csv'
    behavData = pd.read_csv(csvFile, header=None)
    
    # Split the file in PC, mean RT and SD RT
    # Flatten each subject's values for ease of programming
    pCs[:,si] = np.array(behavData[0:4]).flatten()
    mRTs[:,si] = np.array(behavData[4:8]).flatten()
    sdRTs[:,si] = np.array(behavData[8:12]).flatten()
    Ns[:,si] = np.array(behavData[12:]).flatten()
    Rs[:,si] = np.round(Ns[:,si] * pCs[:,si])

In [ ]:
print(Rs[:,0])

In [ ]:
epc = np.ones((nPar, len(x), 20, 9)) * -9
ert = np.ones((nPar, len(x), 20, 9)) * -9
Lp = np.ones(20) * -9
Lrt = np.ones(20) * -9

for si in range(1):
    for pi in xrange(nPar):
        for cdi in range(20):
            # Calculate expected accuracy for each coherence-distance combination
            epc = 1 / (1 + np.exp(-2 * As[pi] * ks[pi] * abs(x)))
            #print(epc)
            # Calculate likelihood of accuracy for this CD combination
            Lp[cdi] = factorial(
            
            # Calculate expected mean RT for each coherence-distance combination
            ert = As[pi] / (ks[pi] * x) * np.tanh(As[pi] * ks[pi] * x) + tRs[pi] 
            #print(ert)
            errs[pi,:,cdi,si] = abs(epc - pCs[cdi,si]) + abs(ert - mRTs[cdi,si])
            #print(pCs[cdi,si],mRTs[cdi,si],errs[pi,:,cdi,si])

Object `fact` not found.


In [ ]:
plt.plot(errs[:,:,:,0].flatten())

In [ ]:
for si in range(1):
    plt.figure()
    for cdi in range(20):
        plt.subplot(4,5,cdi+1)
        plt.contourf(errs[...,cdi,si])

In [ ]:
np.min(errs[...,0])

In [ ]:
plt.contourf(errs[...,4])

In [ ]:
plt.contourf(errs[...,18])

In [ ]:
sortedParams.shape

In [ ]:
sortedParams = np.zeros((len(As)*len(x),5,20))

for i in range(20):
    flatErrs = np.ravel(errs[...,i])
    idArr = np.argsort(flatErrs)
    sortedParams[:,0,i] = flatErrs[idArr]
    sArr = np.unravel_index(idArr, (len(As),len(x)))
    sortedParams[:,1,i] = As[sArr[0][:]]
    sortedParams[:,2,i] = ks[sArr[0][:]]
    sortedParams[:,3,i] = tRs[sArr[0][:]]
    sortedParams[:,4,i] = x[sArr[1][:]]

In [ ]:
plt.subplot(1,5,1)
plt.imshow(sortedParams[0:19,0,:])
plt.title('Error')
plt.subplot(1,5,2)
plt.imshow(sortedParams[0:19,1,:])
plt.title('A')
plt.subplot(1,5,3)
plt.imshow(sortedParams[0:19,2,:])
plt.title('k')
plt.subplot(1,5,4)
plt.imshow(sortedParams[0:19,3,:])
plt.title('tR')
plt.subplot(1,5,5)
plt.imshow(sortedParams[0:19,4,:])
plt.title('x')

In [ ]:
plt.subplot(1,5,1)
for i in range(5):
    plt.plot(sortedParams[i,0,:],'o-')
plt.title('Error')
plt.subplot(1,5,2)
for i in range(5):
    plt.plot(sortedParams[i,1,:],'o-')
plt.title('A')
plt.subplot(1,5,3)
for i in range(5):
    plt.plot(sortedParams[i,2,:],'o-')
plt.title('k')
plt.subplot(1,5,4)
for i in range(5):
    plt.plot(sortedParams[i,3,:],'o-')
plt.title('tR')
plt.subplot(1,5,5)
for i in range(5):
    plt.plot(sortedParams[i,4,:],'o-')
plt.title('x')

In [ ]:
np.max(errs)